In [41]:
import os
import pandas as pd
from datetime import datetime
import numpy as np
import nltk
import torch
from io import BytesIO

from tqdm.notebook import tqdm

import pandas as pd
import torch

from transformers import BertTokenizer, BertModel
from transformers import AutoModel, AutoTokenizer


# Ensure that the Punkt Tokenizer Models are downloaded
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
#download_dir = 'D:\\downloads'
download_dir = 'D:\\downloads\\amazon_customer_reviews'
print(download_dir)

D:\downloads\amazon_customer_reviews


In [6]:
download_files = os.listdir(download_dir)
print(download_files)

['Reviews.csv']


In [11]:

# Initialize an empty list to store the data
data = []

# Populate the list with file information
for file in download_files:
    filepath = os.path.join(download_dir, file)
    filename, file_extension = os.path.splitext(file)
    data.append({'filepaths': filepath, 'extensions': file_extension, 'filenames': file})

# Create DataFrame from the list
files_df = pd.DataFrame(data)

# Display the first few rows
print(files_df.head())

                                          filepaths extensions    filenames
0  D:\downloads\amazon_customer_reviews\Reviews.csv       .csv  Reviews.csv


In [12]:
# Filter the DataFrame for PDF files
csv_files_df = files_df[files_df['extensions'] == '.csv']

# Create a list of file paths for PDF files
csv_filepaths = csv_files_df['filepaths'].tolist()

# Display the list
print(csv_filepaths)

['D:\\downloads\\amazon_customer_reviews\\Reviews.csv']


In [13]:
reviews_file_path = csv_filepaths[0]
print(reviews_file_path)

D:\downloads\amazon_customer_reviews\Reviews.csv


In [32]:
# Load the CSV file into a DataFrame
df_paragraph = pd.read_csv(reviews_file_path)
df_paragraph.rename(columns={'Text': 'Paragraph'}, inplace=True)
# Display the first few rows of the DataFrame
print(df_paragraph.head())

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                          Paragraph  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [33]:
def split_paragraphs_into_sentences(df):
    # Create a list to store the new rows
    new_rows = []

    # Iterate through each row in the input DataFrame
    for index, row in df.iterrows():
        # Use nltk to split the paragraph into sentences
        sentences = nltk.tokenize.sent_tokenize(row['Paragraph'])

        # Add each sentence as a new row, keeping other columns the same
        for sentence_number, sentence in enumerate(sentences, start=1):
            new_row = row.to_dict()
            new_row['P_index'] = f'P_{index}'
            new_row['S_sentence_number'] = f'S_{sentence_number}'
            new_row['Sentence'] = sentence
            # Ensure 'Paragraph' column is not duplicated
            del new_row['Paragraph']
            new_rows.append(new_row)

    # Create a new DataFrame with the new rows
    df_sentence = pd.DataFrame(new_rows)

    return df_sentence

# Example usage:
# df_paragraph = pd.DataFrame({'Paragraph': ["Your paragraphs here."], ...other columns...})
# df_sentence = split_paragraphs_into_sentences(df_paragraph)

In [31]:
def split_paragraphs_into_sentences_1(df):
    # Create a list to store the new rows
    new_rows = []

    # Iterate through each row in the input DataFrame
    for index, row in df.iterrows():
        # Use nltk to split the paragraph into sentences
        sentences = nltk.tokenize.sent_tokenize(row['Text'])

        # Add each sentence as a new row, keeping other columns the same
        for sentence_number, sentence in enumerate(sentences, start=1):
            new_row = row.to_dict()            
            new_row['P_index'] = f'P_{index}'
            new_row['S_sentence_number'] = f'S_{sentence_number}'
            new_row['Sentence'] = sentence
            new_rows.append(new_row)

    # Create a new DataFrame with the new rows
    df_sentence = pd.DataFrame(new_rows)

    return df_sentence

# Example usage:
# df_paragraph = pd.DataFrame({'Text': ["Your paragraphs here."], ...other columns...})
# df_sentence = split_paragraphs_into_sentences(df_paragraph)


In [34]:
df_sentence = split_paragraphs_into_sentences(df_paragraph)

In [35]:
df_sentence.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'P_index',
       'S_sentence_number', 'Sentence'],
      dtype='object')

In [36]:
df_sentence.iloc[0]['Sentence']

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality.'

In [24]:
# Initialize MinIO client and BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


In [66]:
def generate_vectors(text):
    # Check if GPU is available and use it; otherwise, use CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Send model to device (GPU or CPU)
    model.to(device)

    # Ensure no gradient calculations
    with torch.no_grad():
        # Prepare inputs and send them to the device
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        
        # Forward pass, send model outputs back to CPU
        outputs = model(**inputs).last_hidden_state.mean(dim=1).to('cpu')

    return outputs.numpy()


In [75]:
# progress bar
#from tqdm._tqdm_notebook import tqdm_notebook
from tqdm.notebook import tqdm
tqdm_notebook.pandas(desc="processing")


#df_sentence['Summary_vector'] = df_sentence['Summary'].apply(lambda x: generate_vectors(x) if isinstance(x, str) else np.nan)
df_sentence['Summary_vector'] = df_sentence['Summary'].progress_apply(lambda x: generate_vectors(x) if isinstance(x, str) else np.nan)

processing:   0%|          | 0/2832806 [00:00<?, ?it/s]

In [76]:
df_sentence['Sentence_vector'] = df_sentence['Sentence'].progress_apply(lambda x: generate_vectors(x) if isinstance(x, str) else np.nan)


processing:   0%|          | 0/2832806 [00:00<?, ?it/s]

In [77]:
df_pickle_filename = os.path.join(download_dir,"amazon_reviews_pickle_sentences")
df_sentence.to_pickle(df_pickle_filename)

In [86]:
df_sentence.sample(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,P_index,S_sentence_number,Sentence,Summary_vector,Sentence_vector
453152,90507,B000EFFTR0,AZNB1YMYG0JPT,John E. Tipps,0,0,5,1321574400,Hamburger Helper Potato Stroganoff,P_90506,S_7,!,"[[0.031706095, -0.29236707, 0.08684268, 0.0522...","[[0.24765381, -0.117233835, -0.2429299, -0.216..."
789314,157004,B001GINOQC,A3XG8M5E5ALNP,CG in NC,0,0,5,1284940800,Great choice for a snack,P_157003,S_2,"Everyone I have had try them, loves them!","[[0.0770887, -0.1289951, 0.25035286, 0.1587796...","[[0.5010618, 0.3455162, 0.17572194, -0.2641657..."
221269,44662,B001EQ55RW,A3QVAKVRAH657N,"Orrin C. Judd ""brothersjudddotcom""",0,0,5,1211587200,EVEN THE WIFE WAS HAPPY,P_44661,S_7,These latter are most notable because they man...,"[[-0.10866005, -0.19496475, 0.19317056, 0.1024...","[[-0.17071877, 0.21959276, 0.35957518, 0.11829..."
2068171,413742,B0026KPDG8,A3TQ0ZSAD2G98A,Madeline C. Kimmich,0,0,5,1319846400,Salt and Pepper!!!,P_413741,S_2,We bought a variety pack.,"[[0.06598242, 0.23023576, 0.09852357, -0.48538...","[[0.71364367, -0.44887614, -0.012389574, 0.369..."
500777,99744,B003I6OEMI,A1AOS6DFYSOGC4,Jcallie,0,0,5,1282521600,Awesome Thomas Pez Dispenser,P_99743,S_1,Awesome Thomas pez dispensers.,"[[0.061932158, -0.10549656, 0.1725223, 0.29008...","[[0.08059786, -0.054100454, 0.014445701, 0.310..."


In [82]:
import sys

# Assuming df is your DataFrame
size_in_bytes = sys.getsizeof(df_sentence)
size_in_mb = size_in_bytes / (1024**2)  # Convert to Megabytes
print("DataFrame size: {:.2f} MB".format(size_in_mb))


DataFrame size: 2356.73 MB


In [94]:
import gc
gc.collect()


665

In [95]:
import sys

# Create a list of tuples from the global symbol table to avoid RuntimeError
all_objects = [(name, sys.getsizeof(obj)) for name, obj in globals().items()]

# Sort the list by size
sorted_objects = sorted(all_objects, key=lambda x: x[1], reverse=True)

# Print the object names and their sizes
for name, size in sorted_objects:
    print(f"{name}: {size} bytes")


df_sentence: 2471212020 bytes
_78: 2471212020 bytes
_20: 1700325293 bytes
df_paragraph: 467442561 bytes
csv_df: 467424279 bytes
_86: 4216 bytes
BertTokenizer: 2008 bytes
BertModel: 2008 bytes
tqdm: 2008 bytes
tqdm_notebook: 2008 bytes
_28: 1883 bytes
all_objects: 1656 bytes
sorted_objects: 1528 bytes
___: 1464 bytes
_89: 1464 bytes
_i33: 1119 bytes
_35: 1118 bytes
_27: 1114 bytes
_i16: 1090 bytes
_i63: 1090 bytes
AutoModel: 1064 bytes
AutoTokenizer: 1064 bytes
_i31: 1028 bytes
_i62: 1015 bytes
_80: 981 bytes
_i54: 947 bytes
_i57: 947 bytes
_i58: 942 bytes
_i52: 941 bytes
_i59: 941 bytes
_i42: 940 bytes
_i46: 934 bytes
_i44: 933 bytes
_ih: 920 bytes
In: 920 bytes
_i40: 822 bytes
_i25: 675 bytes
_i66: 675 bytes
_oh: 640 bytes
Out: 640 bytes
_i11: 484 bytes
csv_files_df: 476 bytes
files_df: 476 bytes
_i3: 472 bytes
_i7: 472 bytes
_i75: 445 bytes
_i83: 436 bytes
_iii: 422 bytes
_i84: 422 bytes
_i88: 422 bytes
_i92: 422 bytes
_i95: 422 bytes
_i41: 420 bytes
_i70: 411 bytes
_i71: 409 bytes
d

In [91]:
for element in sorted_objects[0:4]:
    print(element)


('___', 2471212020)
('df_sentence', 2471212020)
('_78', 2471212020)
('_20', 1700325293)


In [87]:
print(__)  
print(type(__))  


Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Paragraph'],
      dtype='object')
<class 'pandas.core.indexes.base.Index'>


In [ ]:
#del df_sentences


In [ ]:
gc.collect()

In [80]:
df_paragraph.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Paragraph'],
      dtype='object')

In [96]:
df_paragraph['Paragraph_vector'] = df_paragraph['Paragraph'].progress_apply(lambda x: generate_vectors(x) if isinstance(x, str) else np.nan)

processing:   0%|          | 0/568454 [00:00<?, ?it/s]

In [97]:
df_pickle_filename = os.path.join(download_dir,"amazon_reviews_pickle_paragraphs")
df_paragraph.to_pickle(df_pickle_filename)

In [ ]:
df_xlsx_filename = os.path.join(download_dir,"amazon_reviews_pickle_paragraphs.xlsx")
df_paragraph.to_excel(df_xlsx_filename, index=False)

In [98]:
stop

NameError: name 'stop' is not defined

In [ ]:
import pandas as pd
from chromadb import Chroma

# Create a sample DataFrame
df = pd.DataFrame({
    'Id': [1, 2, 3],
    'ProductId': ['B0001PB9FE', 'B0001PB9FE', 'B0001PB9FE'],
    'UserId': ['A3SGXH7AUHU8GW', 'A1D87F6ZCVE5NK', 'ABXLMWJIXXAIN'],
    'ProfileName': ['delmartian', 'dll pa', 'Natalia Corres "Naty Corres"'],
    'HelpfulnessNumerator': [1, 0, 1],
    'HelpfulnessDenominator': [1, 0, 1],
    'Score': [5, 1, 4],
    'Time': [1303862400, 1346976000, 1219017600],
    'Summary': ['Good Quality Dog Food', 'Not as Advertised', 'Delightful'],
    'P_index': [0, 1, 2],
    'S_sentence_number': [0, 1, 2],
    'Sentence': ['I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.', 'Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".', 'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.']
    'Summary_vector': [[0.1, 0.2, ..., 0.3], [0.4, 0.5, ..., 0.6], [0.7, 0.8, ..., 0.9]],
    'Sentence_vector': [[0.1, 0.2, ..., 0.3], [0.4, 0.5, ..., 0.6], [0.7, 0.8, ..., 0.9]]
})

# Convert the DataFrame into a list of dictionaries
dicts = df.to_dict(orient='records')

# Convert the list of dictionaries into a list of vectors
vectors = [d['Summary_vector'] + d['Sentence_vector'] for d in dicts]

# Store the list of vectors in ChromaDB
chroma = Chroma()
chroma.put(vectors)


In [ ]:
stop

In [ ]:
import pandas as pd
from chromadb import Chroma

# Create a sample DataFrame
df = pd.DataFrame({
    'Id': [1, 2, 3],
    'ProductId': ['B0001PB9FE', 'B0001PB9FE', 'B0001PB9FE'],
    'UserId': ['A3SGXH7AUHU8GW', 'A1D87F6ZCVE5NK', 'ABXLMWJIXXAIN'],
    'ProfileName': ['delmartian', 'dll pa', 'Natalia Corres "Naty Corres"'],
    'HelpfulnessNumerator': [1, 0, 1],
    'HelpfulnessDenominator': [1, 0, 1],
    'Score': [5, 1, 4],
    'Time': [1303862400, 1346976000, 1219017600],
    'Summary': ['Good Quality Dog Food', 'Not as Advertised', 'Delightful'],
    'P_index': [0, 1, 2],
    'S_sentence_number': [0, 1, 2],
    'Sentence': ['I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.', 'Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".', 'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.']
    'Summary_vector': [[0.1, 0.2, ..., 0.3], [0.4, 0.5, ..., 0.6], [0.7, 0.8, ..., 0.9]],
    'Sentence_vector': [[0.1, 0.2, ..., 0.3], [0.4, 0.5, ..., 0.6], [0.7, 0.8, ..., 0.9]]
})

# Convert the DataFrame into a list of dictionaries with metadata
dicts = df.to_dict(orient='records')

# Convert the list of dictionaries into a list of vectors with metadata
vectors = [{'vector': d['Summary_vector'] + d['Sentence_vector'], 'metadata': d} for d in dicts]

# Store the list of vectors in ChromaDB
chroma = Chroma()
chroma.put(vectors)
